In [1]:
import random
import numpy as np
import os

# Initialization of the population

In [2]:
def initialize_pop(pop_size, dimension, low_bound, up_bound):
    """ Function that initialize a population with random (uniform distribution) data between bounds"""
    pop = np.empty([pop_size, dimension])
    for j in range(pop_size):
        for i in range(dimension):
            pop[j][i]= random.uniform(low_bound, up_bound)
    return pop

# Selection of the Best Candidates

In [3]:
def select_mating_pool(pop, fitness, num_parents, maximization = False):
    """Function which isolate the n better candidate (parents) from the population ('best', based on fitness) to generate offspring
    Arguments: 
        pop: a population of size n and dimension m
        fitness: result of objective function => a np.array of value of size n
        num_parents: number of parents that you keep for next generation
        maximization: if False, the probleme is a minimization problem """
    
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = np.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        # Search the higher fi
        if maximization == False:
            fitness_idx = np.where(fitness == np.min(fitness))
            fitness_idx = fitness_idx[0][0]
            fitness[fitness_idx] = 99999999999
        else :
            fitness_idx = np.where(fitness == np.max(fitness))
            fitness_idx = fitness_idx[0][0]
            fitness[fitness_idx] = -99999999999
            
        parents[parent_num, :] = pop[fitness_idx, :]
        
    return parents

# Crossover

In [4]:
def crossover(parents, offspring_size, crossover_type = 'SNP', crossover_rate = 0.5):
    """
    A Function that perform crossover (exchange of value), between 2 random parents
    Arguments:
        parents: result of the function 'select_mating_pool'. A liste of candidates selected.
        offspring_size: size of the output population
        crossover_rate: probability (that a crossover occur)
        crossover_type: type of crossover 'SNP' : single 'gene' modification
                                        ,'segment': 
    """
    offspring = np.empty([offspring_size, parents.shape[1]])
    
    # Keep intact the parents
    for i in range(5):
        offspring[i]=parents[i]
    
    
    for k in range(offspring_size-parents.shape[0]):
        # Choose randomly the parents to crossover
        parent1_idx = random.randint(0, parents.shape[0]-1)
        parent2_idx = random.randint(0, parents.shape[0]-1)
        
        # Crossover of type 'SNP' (The crossover is performed for each 'gene')
        if crossover_type == 'SNP':
            for i in range(parents.shape[1]):
                if np.random.binomial(1, crossover_rate) == 1:  
                    offspring[k+5, i] = parents[parent1_idx, i]
                else:
                    offspring[k+5, i] = parents[parent2_idx, i]
                    
        # Crossover of type 'segment' (many successive 'genes' are exchange
        else:
            # The point at which crossover takes place (here between two parents)
            crossover_point = np.int(parents.shape[1]/2)
            for i in range(parents.shape[1]):
                offspring[k+5, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
                offspring[k+5, crossover_point+1:] = parents[parent2_idx, crossover_point+1:]
                
    return offspring

# Mutation

In [5]:
def mutation(offspring, mutation_type= 'gaussian', mutation_rate= 0.9, mutation_intensity = 0.01, low_bound = -10, up_bound = 10):

    # Mutation changes a single gene in each offspring randomly.
    for j in range(offspring.shape[0]-1):
        for i in range(offspring.shape[1]):
            
            # Gaussian mutation intensity with binomial mutation probablity
            if np.random.binomial(1, mutation_rate) == 1:  
                # The random value to be added to the gene.
                random_value = np.random.normal(0, mutation_intensity)
                
                if  (offspring[j+1, i] + random_value) < low_bound:
                    offspring[j+1, i] = low_bound
                    
                elif (offspring[j+1, i] + random_value) > up_bound:
                    offspring[j+1, i] = up_bound
               
                else:
                    offspring[j+1, i] += random_value
                    
            else: 
                pass

    return offspring

In [11]:
def GA(function = 'Rastrigin'
                    ,pop_size = 100
                   ,dimension = 10
                   ,lower_bound = -10
                   ,upper_bound = 10
                   ,iteration = 250
                   ,stop_rule = 0.001
                   ,num_parents = 5
                   ,maximization = False
                   ,offspring_size = 100
                   ,crossover_rate = 0.5
                   ,crossover_type = 'SNP'
                   ,mutation_intensity = 0.01
                   ,mutation_rate = 0.9
                   ,mutation_type = 'gaussian'
                   ,mutation_decrease_time = 10
                   ,mutation_decrease_intensity = 0.25
                   ,mutation_jump = 50
                  ):
    
    original_path = os.getcwd()
    os.chdir('../problem')
    from ipynb.fs.full.objective_function import objective_function
    from ipynb.fs.full.target_bias import target_bias
    os.chdir(original_path)
    
    #Initialisation of the population
    pop = initialize_pop(pop_size, dimension=dimension, low_bound = lower_bound, up_bound = upper_bound)
    
    # Variable initialisation
    count, count2 = 0,0
    best_fitness_log = np.empty(iteration)
    
    for i in range(iteration):
        
        # Calcul the fitness of the objective function. Best of the population is saved
        fitness = objective_function(pop = pop
                                     ,function = function
                                     ,dimension = dimension)
        
        best_fitness_log[i] = np.min(fitness)
        
        # Display count
        if i % 100 == 0:
            print('Current iteration: {}. fitness: {}'.format(i, np.min(fitness)))
        
        # Parents selection. (Note: the parents are sorted from 'best' to 'worst'
        parents = select_mating_pool(pop = pop
                                     ,fitness = fitness
                                     ,num_parents = num_parents
                                     ,maximization = False)
        
        # If the 'best' parent (based on fitness) is better that the previous best, it is replace
        if i == 0:
            best_candidate = parents[0]
        elif  best_fitness_log[i] < best_fitness_log[i-1]:
            best_candidate = parents[0]

        # Offpsring creation
        offspring = crossover(parents = parents
                              ,offspring_size = offspring_size
                              ,crossover_rate = crossover_rate
                              ,crossover_type = crossover_type)
        
        # Counting of iteration since no observation were observed
        if best_fitness_log[i] == best_fitness_log[i-1]:
            count += 1
            count2 +=1
            
            # If no evolution after n iteration, decrease mutation intensity
            if count == mutation_decrease_time:
                mutation_intensity = mutation_intensity*mutation_decrease_intensity
                count = 0
            
            # If no evolution after n iteration, rise mutation intensity to augment variability
            elif count2 == mutation_jump:
                mutation_intensity = 1
        
        # Mutation of the offspring
        pop = mutation(offspring = offspring
                       ,mutation_rate= 0.9
                       ,mutation_intensity = 0.01
                       ,mutation_type= 'gaussian'
                       ,low_bound = lower_bound
                       ,up_bound = upper_bound)
        
        # Stopping criterion
        if best_fitness_log[i] <= stop_rule:
            print('Stop by stoping rule at iteration {}. Best fitness: {}. Stop rule: {}'.format(i, best_fitness_log[i], stop_rule))
            break

    return best_fitness_log[:i+1], best_candidate